# Enhanced Secret Sharing Scheme (ESSS)
This notebook contains cryptographic functions, helper utilities, and interpolation methods for secure secret sharing.

## Cryptographic Setup
This section contains necessary imports for cryptographic operations using Python's `pycryptodome` library.

In [1]:
from Crypto.Cipher import DES, DES3, AES, ARC4, CAST
from Crypto.Random import get_random_bytes
from Crypto.Util.Padding import pad, unpad
import random

## Helper Functions

This section contains the essential utility functions used throughout the program. These functions are designed to perform specific tasks, such as conversions, mathematical computations, encryption, decryption, or support for the main algorithm. They help in modularizing the code, making it more readable, reusable, and easier to maintain.

### List of Helper Functions:
1. **`encryption`**: Encrypts the given data using a specified encryption algorithm or technique.
2. **`decryption`**: Decrypts the encrypted data back to its original form.
3. **`byte_to_decimal`**: Converts a bytes object into its decimal integer representation.
4. **`decimal_to_byte`**: Converts a decimal integer into its bytes representation.
5. **`lagrange_interpolation`**: Reconstructs the secret or evaluates a polynomial at a given point using Lagrange Interpolation from the provided points (shares).
6. **`get_polynomial`**: Generates the coefficients of a polynomial with a secret as the constant term.
7. **`get_uniqueID_and_share`**: Generates unique IDs and their corresponding shares using a polynomial.


These functions act as building blocks and are integral to the execution of the main logic.

In [2]:
def encryption(data, encryption_func='DES'):
    """
    Encrypts the given data using the specified encryption algorithm.

    Parameters:
        data (bytes): The data to be encrypted.
        encryption_func (str): The encryption algorithm to use (e.g., "DES", "3DES", "AES", "ARC4", "CAST").

    Returns:
        bytes: The encrypted data.
    """
    
    # Define keys and initialization vectors (IVs) for different encryption algorithms
    key_des = b'`\xc9\x98\xa6\x0f\x99@\x0f'  # DES requires an 8-byte key
    key_3des = b'\xbf^\xcd\xda]#\xae\xad@\xdc\x16R\x10@pWkL\x13\xb98\x918\xf8'  # 3DES requires a 24-byte key
    key_aes = b'\xb6\x89\xfa\xaf\xb4\xa7\x91b\xdex\x8b\x1a\xca~@-'  # AES requires a 16, 24, or 32-byte key
    key_arc4 = b'\xf5,MO\xed\x95!\xdd\xb1\x17\xb4u\x15I\xce\x12'  # ARC4 key can be 1 to 256 bytes
    key_cast = b'Q\x1a\xf8\r\x96\x96\xffa_[\xd4\x8dk\xce\x12\xba'  # CAST128 key can be 5 to 16 bytes
    
    iv_des_3des_cast = b'g\x03\x95\x86\x13I\x89\x9c'  # IV for DES, 3DES, and CAST128 (8 bytes)
    iv_aes = b'5\x81\xc5\xa6\x8d\xda\x19V\x83\x7f\xa9E\x98\x94z\xc1'  # IV for AES (16 bytes)

    # DES
    if encryption_func == "DES":
        cipher = DES.new(key_des, DES.MODE_CBC, iv_des_3des_cast)
        encrypted_data = cipher.encrypt(pad(data, DES.block_size))
        
    # 3DES
    elif encryption_func == "3DES":
        cipher = DES3.new(key_3des, DES3.MODE_CBC, iv_des_3des_cast)
        encrypted_data = cipher.encrypt(pad(data, DES3.block_size))
        
    # AES
    elif encryption_func == "AES":
        cipher = AES.new(key_aes, AES.MODE_CBC, iv_aes)
        encrypted_data = cipher.encrypt(pad(data, AES.block_size))
    
    # ARC4
    elif encryption_func == "ARC4":
        cipher = ARC4.new(key_arc4)
        encrypted_data = cipher.encrypt(data)
        
    # CAST128
    elif encryption_func == "CAST": 
        cipher = CAST.new(key_cast, CAST.MODE_CBC, iv_des_3des_cast)
        encrypted_data = cipher.encrypt(pad(data, CAST.block_size))
    else:
        raise ValueError(f"Unsupported encryption method: {encryption_func}")

    return encrypted_data

In [3]:
def decryption(data, decryption_func='DES'):
    """
    Decrypts the given data using the specified decryption algorithm.

    Parameters:
        data (bytes): The data to be decrypted.
        decryption_func (str): The decryption algorithm to use (e.g., "DES", "3DES", "AES", "ARC4", "CAST").

    Returns:
        bytes: The decrypted data.
    """
    
    # Define keys and initialization vectors (IVs) for different encryption algorithms
    key_des = b'`\xc9\x98\xa6\x0f\x99@\x0f'  # DES requires an 8-byte key
    key_3des = b'\xbf^\xcd\xda]#\xae\xad@\xdc\x16R\x10@pWkL\x13\xb98\x918\xf8'  # 3DES requires a 24-byte key
    key_aes = b'\xb6\x89\xfa\xaf\xb4\xa7\x91b\xdex\x8b\x1a\xca~@-'  # AES requires a 16, 24, or 32-byte key
    key_arc4 = b'\xf5,MO\xed\x95!\xdd\xb1\x17\xb4u\x15I\xce\x12'  # ARC4 key can be 1 to 256 bytes
    key_cast = b'Q\x1a\xf8\r\x96\x96\xffa_[\xd4\x8dk\xce\x12\xba'  # CAST128 key can be 5 to 16 bytes
    
    iv_des_3des_cast = b'g\x03\x95\x86\x13I\x89\x9c'  # IV for DES, 3DES, and CAST128 (8 bytes)
    iv_aes = b'5\x81\xc5\xa6\x8d\xda\x19V\x83\x7f\xa9E\x98\x94z\xc1'  # IV for AES (16 bytes)

    # DES
    if decryption_func == "DES":
        cipher = DES.new(key_des, DES.MODE_CBC, iv_des_3des_cast)
        decrypted_data = unpad(cipher.decrypt(data), DES.block_size)
        
    # 3DES
    elif decryption_func == "3DES":
        cipher = DES3.new(key_3des, DES3.MODE_CBC, iv_des_3des_cast)
        decrypted_data = unpad(cipher.decrypt(data), DES3.block_size)
        
    # AES
    elif decryption_func == "AES":
        cipher = AES.new(key_aes, AES.MODE_CBC, iv_aes)
        decrypted_data = unpad(cipher.decrypt(data), AES.block_size)
        
    # ARC4
    elif decryption_func == "ARC4":
        cipher = ARC4.new(key_arc4)
        decrypted_data = cipher.decrypt(data)
        
    # CAST128
    elif decryption_func == "CAST": 
        cipher = CAST.new(key_cast, CAST.MODE_CBC, iv_des_3des_cast)
        decrypted_data = unpad(cipher.decrypt(data), CAST.block_size)

    else:
        raise ValueError(f"Unsupported encryption method: {encryption_func}")

    return decrypted_data

In [4]:
# Convert bytes to decimal
def byte_to_decimal(byte_data):
    """
    Converts a bytes object to its decimal integer representation.

    Parameters:
        byte_data (bytes): The byte data to convert.

    Returns:
        int: The decimal integer representation of the byte data.
    """
    decimal_value = int.from_bytes(byte_data, byteorder='big')
    return decimal_value
    
# Convert decimal to bytes
def decimal_to_byte(decimal_value):
    """
    Converts a decimal integer to its bytes representation.

    Parameters:
        decimal_value (int): The decimal integer to convert.

    Returns:
        bytes: The bytes representation of the decimal integer.
    """
    byte_data = decimal_value.to_bytes((decimal_value.bit_length() + 7) // 8, byteorder='big')
    return byte_data

In [5]:
def lagrange_interpolation(x_points, y_points, x = 0):
    """
    Perform Lagrange Interpolation.

    Parameters:
        x_points: List of x-coordinates of the data points
        y_points: List of y-coordinates of the data points
        x: The value at which the interpolated polynomial is evaluated

    Returns:
        The interpolated value at x
    """
    n = len(x_points)
    result = 0

    for i in range(n):
        # Compute L_i(x)
        L_i = 1
        for j in range(n):
            if i != j:
                L_i *= (x - x_points[j]) / (x_points[i] - x_points[j])
        
        # Add contribution of y_i * L_i(x)
        result += y_points[i] * L_i

    return result

In [6]:
def get_polynomial(secret, k):
    """
    Generates the coefficients of a polynomial with a given secret as the constant term.

    Parameters:
        secret (int): The secret value to be used as the constant term of the polynomial.
        k (int): The degree of the polynomial plus one (total number of coefficients, including the constant term).

    Returns:
        list: A list of integers representing the coefficients of the polynomial, 
              where the last coefficient is the secret.
    """
    coefficients = [random.randint(1, 100) for _ in range(k-1)]
    coefficients.append(secret)
    return coefficients

# For Example
get_polynomial(1234,5)

[14, 47, 67, 7, 1234]

In [7]:
def get_uniqueID_and_share(secret, k, n):
    """
    Generates unique IDs and their corresponding shares based on a secret and polynomial coefficients.

    This function uses the concept of Shamir's Secret Sharing scheme to generate `n` unique shares 
    from a polynomial of degree `k-1`, with the secret as the constant term.

    Parameters:
        secret (int): The secret value to be shared.
        k (int): The minimum number of shares required to reconstruct the secret.
        n (int): The total number of shares to generate.

    Returns:
        tuple: A tuple containing:
            - unique_ids (list): A list of unique IDs (x-values) for the shares.
            - shares (list): A list of corresponding shares (y-values) computed from the polynomial.
    """
    
    cofficients = get_polynomial(secret, k)
    unique_ids = [x for x in range(1,1+n)]
    degree = k-1
    shares = []
    for unique_id in unique_ids:
        share = 0
        for c in cofficients:
            share += c * (unique_id**degree)
            degree -= 1
        shares.append(share)
        degree = k-1
    return unique_ids, shares
    
# For Example
unique_ids, shares = get_uniqueID_and_share(1234, 5, 10)
print(unique_ids)
print(shares)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1468, 3094, 9136, 24610, 56524, 113878, 207664, 350866, 558460, 847414]


# Encryption Stage

In [8]:
# Get number of member 
n = int(input("Enter number of members: "))

# Get minimum number of members required to generate secret Key
k = int(input("Enter number of members required to generate secret: "))

#Get Secret Key
secret = int(input("Enter Secret Key: "))

unique_ids, shares = get_uniqueID_and_share(secret, k, n)

Enter number of members:  10
Enter number of members required to generate secret:  5
Enter Secret Key:  1234


In [9]:
# Get encryption function
encryption_func_value = int(input("Select an Encryption Function:\n 1. DES\n 2. 3DES\n 3. AES\n 4. ARC4\n 5. CAST\n "))

if encryption_func_value == 1:
    encryption_func = "DES"
    print("Selected function: DES")
elif encryption_func_value == 2:
    encryption_func = "3DES"
    print("Selected function: 3DES")
elif encryption_func_value == 3:
    encryption_func = "AES"
    print("Selected function: AES")
elif encryption_func_value == 4:
    encryption_func = "ARC4"
    print("Selected function: ARC4")
elif encryption_func_value == 5:
    encryption_func = "CAST"
    print("Selected function: CAST")
else:
    print("Incorrect Function Type!!")

Select an Encryption Function:
 1. DES
 2. 3DES
 3. AES
 4. ARC4
 5. CAST
  1


Selected function: DES


In [10]:
# Convert ids and shares from decimal to bytes
bytes_unique_ids = [decimal_to_byte(unique_id) for unique_id in unique_ids]
bytes_shares = [decimal_to_byte(share) for share in shares]
bytes_unique_ids, bytes_shares

([b'\x01',
  b'\x02',
  b'\x03',
  b'\x04',
  b'\x05',
  b'\x06',
  b'\x07',
  b'\x08',
  b'\t',
  b'\n'],
 [b'\x05\xb8',
  b'\x0c\x10',
  b'"<',
  b'XV',
  b'\xc40',
  b'\x01\x81T',
  b'\x02\xb1\x04',
  b'\x04z:',
  b'\x07\t\xa8',
  b'\n\x91\xb8'])

In [11]:
# Encryt Unique Ids and Shares
encrypted_unique_ids = [encryption(unique_id, encryption_func) for unique_id in bytes_unique_ids]
encrypted_shares = [encryption(share, encryption_func) for share in bytes_shares]
encrypted_unique_ids, encrypted_shares

([b'\xc6\x15\x91\x90\xf5\x88\x91\x1f',
  b'H\xbbn\x92P\xa5\xbb\xb1',
  b';r\xf9\xf4\xa1\xde\x11+',
  b'\xa8at9\xba\x0f\xf6\x94',
  b'\xaa2R\x91N+\xe2\xc5',
  b'\x9egm\xd6\x1f\x98\xdd/',
  b'\x10\xe7\xc7\x8a\x8abcJ',
  b'\x0f\xf2+\xd5\x93mt\xb5',
  b'F\xeb~x\xdb\xeaZ\xd2',
  b'\xd6\xfcI\xf7\xc7\xe07f'],
 [b'\x8d\xa7\xfc\x031G\x933',
  b'\x962\x12gpdI\n',
  b'\x82\x0cw\xf8\xbc2a\xc6',
  b'5\xb1`,Q\xabl\x8d',
  b'\xe7\x00 s\xfa\x7fG\xd5',
  b'Z4\xf7\x166\x03\xfa\xef',
  b'\x0c\xc0\xb2l\x87\x03\xa2R',
  b'\xda(\x10\xca\xf0+\x04\x18',
  b'\xd1\xee\x97\x95\xc3T\xb7\x8c',
  b"\xcc\x8eu'\x0b\xd2\xd0s"])

In [12]:
# Convert ids and shares to user readable decimal format
encrypted_unique_ids = [byte_to_decimal(unique_id) for unique_id in encrypted_unique_ids]
encrypted_shares = [byte_to_decimal(share) for share in encrypted_shares]
encrypted_unique_ids, encrypted_shares

([14273474645801341215,
  5240904166073023409,
  4283761024665784619,
  12133106662396393108,
  12263955519318516421,
  11414212547014352175,
  1218161622068126538,
  1149029051234350261,
  5110317259732703954,
  15491338146806773606],
 [10207404171100394291,
  10822733090001275146,
  9370996834844172742,
  3868979298352983181,
  16645339905257326549,
  6500091836970040047,
  918930503174955602,
  15719832963135505432,
  15127194867867105164,
  14739847440992751731])

# Decryption Stage

In [13]:
# Get k Ids and Shares from the user
k_ids = []
k_shares = []
print(f"Enter atleast {k} pairs of unique ids and shares")
for i in range(k):
    k_ids.append(int(input("Enter Id:")))
    k_shares.append(int(input("Enter Corresponding Share:")))
k_ids, k_shares

Enter atleast 5 pairs of unique ids and shares


Enter Id: 14273474645801341215
Enter Corresponding Share: 12569928172220221977
Enter Id: 5240904166073023409
Enter Corresponding Share: 4019553605964139415
Enter Id: 4283761024665784619
Enter Corresponding Share: 17859154192713726847
Enter Id: 12133106662396393108
Enter Corresponding Share: 11519957922990256374
Enter Id: 12263955519318516421
Enter Corresponding Share: 260302386392070680


([14273474645801341215,
  5240904166073023409,
  4283761024665784619,
  12133106662396393108,
  12263955519318516421],
 [12569928172220221977,
  4019553605964139415,
  17859154192713726847,
  11519957922990256374,
  260302386392070680])

In [14]:
# Convert ids and shares to bytes format for decryption
encrypted_unique_ids = [decimal_to_byte(unique_id) for unique_id in k_ids]
encrypted_shares = [decimal_to_byte(share) for share in k_shares]
encrypted_unique_ids, encrypted_shares

([b'\xc6\x15\x91\x90\xf5\x88\x91\x1f',
  b'H\xbbn\x92P\xa5\xbb\xb1',
  b';r\xf9\xf4\xa1\xde\x11+',
  b'\xa8at9\xba\x0f\xf6\x94',
  b'\xaa2R\x91N+\xe2\xc5'],
 [b'\xaeq[\x13\x05\\\x8a\x19',
  b'7\xc8R\xb5G"\x0b\x97',
  b'\xf7\xd8v\x1d\xcd\xa1;\x7f',
  b'\x9f\xdf\x1c\xb2\x13G\xf0\xf6',
  b'\x03\x9c\xc7\xa4+\x15\xfa\x18'])

In [15]:
# Decryt Unique Ids and Shares
decrypted_unique_ids = [decryption(unique_id,  encryption_func) for unique_id in encrypted_unique_ids]
decrypted_shares = [decryption(share, encryption_func) for share in encrypted_shares]
decrypted_unique_ids, decrypted_shares

([b'\x01', b'\x02', b'\x03', b'\x04', b'\x05'],
 [b'\x05\xb2', b'\n\xc2', b'\x1b\xc4', b'Dj', b'\x94V'])

In [16]:
# Convert ids and shares from bytes to decimal
de_unique_ids = [byte_to_decimal(unique_id) for unique_id in decrypted_unique_ids]
de_shares = [byte_to_decimal(share) for share in decrypted_shares]
de_unique_ids, de_shares

([1, 2, 3, 4, 5], [1458, 2754, 7108, 17514, 37974])

In [17]:
# Use Lagrange's Interpolation for Secret Generation
secret = lagrange_interpolation(de_unique_ids, de_shares)
print(f"Secret is {int(secret)}")

Secret is 1234


### If provided k ids and shares are incorrect then the output secret is incorrect and useless